<a href="https://colab.research.google.com/github/qahathaway/Automated_Segmentation_PTM-IMAT-SAT/blob/main/Pectoralis_Semantic_Segmentation_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install python3.7
!sudo apt-get update -y
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7.1 1
!sudo update-alternatives --config python3 <<< '2'

In [ ]:
!python --version

In [ ]:
!git clone https://github.com/qahathaway/Automated_Segmentation_PTM-IMAT-SAT.git

In [ ]:
!mkdir /content/Automated_Segmentation_PTM-IMAT-SAT/Selected
!mkdir /content/Automated_Segmentation_PTM-IMAT-SAT/Output
!mkdir /content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer

# Step 3: Select the frame directly above the aortic arch

In [ ]:
### Load Essential Packages ###
import numpy as np
import pandas as pd
import os
import cv2
from imutils import paths
import argparse
import glob
from glob import iglob

n_white_pix = []
names = []
file_count = []
index = 0

### Select Frame Directly Above the Aorta ###
for filename in sorted(glob.glob('/content/Automated_Segmentation_PTM-IMAT-SAT/Example/**/axial_*.png'), key=lambda name: (name[53:61], int(os.path.basename(name)[14:-4]))):
    image = cv2.imread(filename)
    names.append(filename)
    pathlen, filenamed = os.path.split(filename)
    file_count = len(glob.glob1(pathlen,"axial_*.png"))
    image[image <= 80] = 0
    image[image >= 140] = 0
    cropped = image[165:360, 180:332]
    n_white_pix = np.append(n_white_pix, np.sum(cropped > 0))
    index += 1

### Select the Array of Images to Search and Apply Pixel Intensities to Stratify ###
    if index == file_count:
        lower = int(file_count*0.6)
        upper = int(file_count*0.85)
        minimum = (np.argmin(n_white_pix[lower:upper]) + lower)
        midline = cv2.imread(names[minimum])
        path, filenames = os.path.split(names[minimum])
        path2 = os.path.basename(path)
        cv2.imwrite('/content/Automated_Segmentation_PTM-IMAT-SAT/Selected/' + path2 + "_" + filenames, midline)

        n_white_pix = []
        names = []
        file_count = []
        index = 0

    else:
        continue


# Step 4: Custom semantic segmentation using Pixellib and Mask R-CNN for all remaining participants

In [ ]:
!sudo apt install python3-pip

In [ ]:
!pip install tensorflow==2.5.3
!pip install tensorflow-gpu==2.5.0
!pip install tensorboard==2.5.0
!pip install keras==2.4.3
!pip install numpy==1.19.5
!pip install pillow==8.2.0
!pip install labelme2coco
!pip install PyQt5==5.15.5
!pip install pixellib==0.7.1
!pip install scikit-image==0.18.0

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Train the MASK R-CNN Model from the Groundtruth Images ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes= 3, batch_size = 12)
train_maskrcnn.load_pretrained_model("/content/Automated_Segmentation_PTM-IMAT-SAT/COCO/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("path/to/manual/segementation")
train_maskrcnn.train_model(num_epochs = 12, augmentation=False,  path_trained_models = "/content/Automated_Segmentation_PTM-IMAT-SAT/COCO/")

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.custom_train import instance_custom_training

### Evaluate performance of the MASK R-CNN Model using the Test/Holdout Group ###
train_maskrcnn = instance_custom_training()
train_maskrcnn.modelConfig(network_backbone = "resnet50", num_classes= 3, batch_size = 1)
train_maskrcnn.load_pretrained_model("/content/Automated_Segmentation_PTM-IMAT-SAT/COCO/mask_rcnn_coco.h5")
train_maskrcnn.load_dataset("path/to/manual/segementation")
train_maskrcnn.evaluate_model("/content/Automated_Segmentation_PTM-IMAT-SAT/COCO/95_MSK_3_Class.h5")

In [ ]:
### Load Essential Packages ###
import pixellib
from pixellib.instance import custom_segmentation

### Custom Segmentation of Left PMAT, Right PMAT, and SAT ###
segment_image = custom_segmentation()
segment_image.inferConfig(num_classes = 3, class_names= ["BG", "PMR", "PML", "SAT"], detection_threshold = 0.9, network_backbone = "resnet50")
segment_image.load_model("/content/Automated_Segmentation_PTM-IMAT-SAT/COCO/95_MSK_3_Class.h5")

In [ ]:
### Load Essential Packages ###
import numpy as np
from PIL import Image
import cv2
import os

### Set Directories ###
inputdir = '/content/Automated_Segmentation_PTM-IMAT-SAT/Selected/'
outdir = '/content/Automated_Segmentation_PTM-IMAT-SAT/Output/'
outdir1 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/PMR_Mask_'
outdir2 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/PMR_'
outdir3 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/PML_Mask_'
outdir4 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/PML_'
outdir5 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/SAT_Mask_'
outdir6 = '/content/Automated_Segmentation_PTM-IMAT-SAT/Mask_Infer/SAT_'

test_list = [ f for f in  os.listdir(inputdir)]

### Inference of Left PMAT, Right PMAT, and SAT Images Based on Mask R-CNN Model ###
for f in sorted (test_list):
    if f.endswith('.png'):
        segmask, output = segment_image.segmentImage(inputdir + f, extract_segmented_objects = False, mask_points_values = True, show_bboxes=False)
        class_list = segmask['class_ids'].tolist()
        class_str = ''.join(str(x) for x in class_list)
        try:
            if class_str == '321':
                c,b,a = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '312':
                c,a,b = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '231':
                b,c,a = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '213':
                b,a,c = [ [individualArray] for individualArray in segmask['masks'] ]
            elif class_str == '132':
                a,c,b = [ [individualArray] for individualArray in segmask['masks'] ]
            else:
                a,b,c = [ [individualArray] for individualArray in segmask['masks'] ]
        except:
            pass

        cv2.imwrite(outdir + f, output)
        a1 = np.array(a)
        b1 = np.array(b)
        c1 = np.array(c)
        for i in a1.tolist():
            mask1 = np.asarray(Image.open(inputdir + f))
            mask11 =  mask1[:,:,:3]
            img1 = cv2.fillPoly(mask1, np.array(i), color=(0, 0, 0))
            maskB = Image.fromarray(mask11)
        for i in a1.tolist():
            mask2 = np.asarray(Image.open(inputdir + f))
            mask22 =  mask2[:,:,:3]
            img2 = cv2.fillPoly(mask2, np.array(i), color=(255, 255, 255))
            maskW = Image.fromarray(mask22)
            ROI_PMR_Mask = np.subtract(maskW, maskB)
            ROI_PMR = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB)
            cv2.imwrite(outdir1 + f, ROI_PMR_Mask)
            cv2.imwrite(outdir2 + f, ROI_PMR)
        for i in b1.tolist():
            mask3 = np.asarray(Image.open(inputdir + f))
            mask33 =  mask3[:,:,:3]
            img3 = cv2.fillPoly(mask3, np.array(i), color=(0, 0, 0))
            maskB2 = Image.fromarray(mask33)
        for i in b1.tolist():
            mask4 = np.asarray(Image.open(inputdir + f))
            mask44 =  mask4[:,:,:3]
            img4 = cv2.fillPoly(mask4, np.array(i), color=(255, 255, 255))
            maskW2 = Image.fromarray(mask44)
            ROI_SAT_Mask = np.subtract(maskW2, maskB2)
            ROI_SAT = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB2)
            cv2.imwrite(outdir3 + f, ROI_SAT_Mask)
            cv2.imwrite(outdir4 + f, ROI_SAT)
        for i in c1.tolist():
            mask5 = np.asarray(Image.open(inputdir + f))
            mask55 =  mask5[:,:,:3]
            img5 = cv2.fillPoly(mask5, np.array(i), color=(0, 0, 0))
            maskB3 = Image.fromarray(mask55)
        for i in c1.tolist():
            mask6 = np.asarray(Image.open(inputdir + f))
            mask66 =  mask6[:,:,:3]
            img6 = cv2.fillPoly(mask6, np.array(i), color=(255, 255, 255))
            maskW3 = Image.fromarray(mask66)
            ROI_PML_Mask = np.subtract(maskW3, maskB3)
            ROI_PML = np.subtract(np.asarray(Image.open(inputdir + f))[:,:,:3], maskB3)
            cv2.imwrite(outdir5 + f, ROI_PML_Mask)
            cv2.imwrite(outdir6 + f, ROI_PML)

Step 6: Determine the total number of pixels within the ROI

In [ ]:
### Load Essential Packages ###
import cv2
import os
import numpy as np
import glob
from glob import iglob
import csv

### Count White Pixels in Image ###
for filename in sorted(glob.glob('path/to/ROI/**/*.png')):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    n_white_pix = np.sum(img == 255)
    average = img[img!=0].mean()
    path, filenames = os.path.split(filename)
    path2 = os.path.basename(path)
    if "Mask" in filenames:
        name_type = "Mask"
    else:
        name_type = "Image"
    if "PML_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Left"
    elif "PMR_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Right"
    elif "PML-PATL_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Left Subtracted from Intermuscular Adipose Tissue, Left"
    elif "PMR-PATR_Mask_" in filenames:
        name_region = "Pectoralis Muscle, Right Subtracted from Intermuscular Adipose Tissue, Right"
    elif "PML-PATL_MESA" in filenames:
        name_region = "Pectoralis Muscle, Left Subtracted from Intermuscular Adipose Tissue, Left"
    elif "PMR-PATR_MESA" in filenames:
        name_region = "Pectoralis Muscle, Right Subtracted from Intermuscular Adipose Tissue, Right"
    elif "EMLL_Mask_" in filenames:
        name_region = "Extramyocellular Lipid, Left"
    elif "EMLR_Mask_" in filenames:
        name_region = "Extramyocellular Lipid, Right"
    elif "SAT_Mask_" in filenames:
        name_region = "Subcutaneous Adipose Tissue"
    elif "PATL_Mask_" in filenames:
        name_region = "Intermuscular Adipose Tissue, Left"
    elif "PATR_Mask_" in filenames:
        name_region = "Intermuscular Adipose Tissue, Right"
    fields=[path2, name_type, name_region, filenames, n_white_pix, average]
    with open('path/to/Measurements_Mask.csv', 'a') as f:
        writer = csv.writer(f)
        writer.writerow(fields)